## TFRecord_0428 학습하기

- 전처리 x
- 증식 x
- row 파일 그대로 TFRecord로 생성

=> 나중에 normalization과 resize 수행  
=> 전체파일크기 맞춰주기는 필요해보임

In [8]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Flatten
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


############### Parameter
NUM_OF_TFRECORDS = 100 # 종류별 TFRecord의 개수
BUFFER_SIZE = 16     # 데이터 shuffle을 위한 buffer size
BATCH_SIZE = 8       # 배치 사이즈. 한번에 가져오는 이미지 데이터 개수 
NUM_CLASS = 88        # class의 개수. binary인 경우는 필요없으며 categorical인 경우 설정
IMAGE_SIZE = 150       


In [9]:
# 학습에 필요한 DataSet 준비(여러개의 tfrecord 처리)

# train, validation TFRecord 폴더 경로(여러개의 tfrecord 처리)
# 폴더를 나누고 파일을 복사하는건 수동으로 처리
train_tfrecord_path = './tfrecords_0428/train'
valid_tfrecord_path = './tfrecords_0428/valid'

train_tfrecord_list = tf.io.gfile.glob(train_tfrecord_path + '/*.tfrecords')
valid_tfrecord_list = tf.io.gfile.glob(valid_tfrecord_path + '/*.tfrecords')

# 읽어들인 TFRecord를 다음의 형태(dict)로 변환하는 함수
# <ParallelMapDataset shapes: {id: (), image_raw: (), label: ()}, 
#                     types: {id: tf.string, image_raw: tf.string, label: tf.int64}>
def _parse_image_function(example_proto):
    # TFRecord를 읽어서 데이터를 복원하기 위한 자료구조.
    image_feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/channel': tf.io.FixedLenFeature([], tf.int64),
        'image/label': tf.io.FixedLenFeature([], tf.int64),
        'image/image_raw': tf.io.FixedLenFeature([], tf.string),
        'image/filename': tf.io.FixedLenFeature([], tf.string)
    }
    return tf.io.parse_single_example(example_proto, 
                                      image_feature_description)

# 위에서 얻은 ParallelMapDataset를 다음의 형태(shape)로 변환하는 함수
# <ParallelMapDataset shapes: ((None, None, 3), ()), types: (tf.float32, tf.int64)>
def map_func(target_record):      
    img = target_record['image/image_raw']
    label = target_record['image/label']
    img = tf.image.decode_jpeg(img, channels=3)    
    return img, label


# 전처리(normalization & resize) 함수
# 이미지 데이터 normalization
# 우리예제는 TFRecord 생성 시 원본 size로 저장했기 때문에 image resize를 해야함.
def image_preprocess_func(image, label):
    result_image = image / 255
    result_image = tf.image.resize(result_image, 
                                   (IMAGE_SIZE,IMAGE_SIZE),
                                   antialias=False)   
    return result_image, label


# 만약 multinomial classification이면 one_hot처리도 필요함.
def image_postprocess_func(image, label):
    onehot_label = tf.one_hot(label, depth=88)    # binary인 경우 one_hot 사용안함.    
    return image, label

def make_dataset(tfrecords_path, is_train):
    
    dataset = tf.data.TFRecordDataset(tfrecords_path)

    dataset = dataset.map(_parse_image_function,
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.map(map_func,
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if is_train:
        dataset = dataset.shuffle(BUFFER_SIZE)

    dataset = dataset.map(image_preprocess_func,
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.map(image_postprocess_func,
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.batch(BATCH_SIZE)

    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

In [12]:
############### Model

input_layer = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='input_layer')  

## Pretrained Network
pretrained_model = VGG16(weights='imagenet',
                         include_top=False,
                         input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                         input_tensor=input_layer)

pretrained_model.trainable = False

x = pretrained_model.output

x = Flatten()(x)
x = Dense(512,
          activation='relu')(x)
x = Dense(88, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=x)


# EarlyStopping & Checkpoint & Learning Rate
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_acc", 
                        patience = 3, 
                        factor = 0.5, 
                        min_lr=1e-7,
                        verbose=1)

model_filename = './checkpoint_0428/ad-{epoch:06d}-{val_acc:0.6f}-{acc:0.6f}.hdf5'

checkpointer = ModelCheckpoint(
    filepath=model_filename, 
    verbose=1, 
    save_freq='epoch', 
    save_best_only=True, 
    monitor='val_acc')

es = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

# LearningRateScheduler 이용
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])



# 데이터 세팅 후 학습(여러개(100개)의 tfrecord를 이용한 학습)
for i in tqdm(range(NUM_OF_TFRECORDS),
              total=NUM_OF_TFRECORDS,
              position=0,
              leave=True):
    tfrecord_train_file = train_tfrecord_list[i]
    tfrecord_valid_file = valid_tfrecord_list[i]
    
    print(tfrecord_train_file)
    
    dataset = make_dataset(tfrecord_train_file, True)
    valid_dataset = make_dataset(tfrecord_valid_file, False)
    print(dataset)
    
    model.fit(dataset,
              epochs=20,
              validation_data=valid_dataset,
#               callbacks = [checkpointer, es, learning_rate_reduction],
              callbacks = [checkpointer],
              verbose=0)       

print('############### 기본학습 종료 ###############')    

  0%|          | 0/100 [00:00<?, ?it/s]

.\tfrecords_0428\train\0_0001_of_0008.tfrecords
<PrefetchDataset shapes: ((None, 150, 150, 3), (None,)), types: (tf.float32, tf.int64)>

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to ./checkpoint_0428\ad-000001-0.000000-0.000000.hdf5

Epoch 00002: val_acc did not improve from 0.00000

Epoch 00003: val_acc did not improve from 0.00000

Epoch 00004: val_acc did not improve from 0.00000

Epoch 00005: val_acc did not improve from 0.00000

Epoch 00006: val_acc did not improve from 0.00000

Epoch 00007: val_acc did not improve from 0.00000

Epoch 00008: val_acc did not improve from 0.00000

Epoch 00009: val_acc did not improve from 0.00000

Epoch 00010: val_acc did not improve from 0.00000

Epoch 00011: val_acc did not improve from 0.00000

Epoch 00012: val_acc did not improve from 0.00000

Epoch 00013: val_acc did not improve from 0.00000

Epoch 00014: val_acc did not improve from 0.00000

Epoch 00015: val_acc did not improve from 0.00000

Epoch 00016: val_acc did not 

KeyboardInterrupt: 

In [ ]:
# 여기까지가 기본학습 처리입니다.

# Pretrained Network 위에 새로운 Network을 추가합니다. 
# Base Network을 동결합니다.
# 새로 추가한 Network을 학습합니다.


# 아래의 작업이 추가로 필요합니다.
# Base Network에서 일부 Layer의 동결을 해제합니다.
# 동결을 해제한 층과 새로 추가한 층을 함께 학습합니다.

pretrained_model.trainable = True

for layer in pretrained_model.layers:
    if layer.name in ['block5_conv1','block5_conv2','block5_conv3']:
        layer.trainable = True
    else:    
        layer.trainable = False

## learning rate를 줄이는게 일반적(미세조절)        

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])
    
    
    
## 재학습 진행
print('########### 재학습 진행 ###########')

for i in tqdm(range(NUM_OF_TFRECORDS),
              total=NUM_OF_TFRECORDS,
              position=0,
              leave=True):

    tfrecord_train_file = train_tfrecord_list[i]
    tfrecord_valid_file = valid_tfrecord_list[i]
    
    dataset = make_dataset(tfrecord_train_file, True)
    valid_dataset = make_dataset(tfrecord_valid_file, False)
    
    model.fit(dataset,
              epochs=20,
              validation_data=valid_dataset,
#               callbacks = [checkpointer, es, learning_rate_reduction],
              callbacks = [checkpointer],
              verbose=0)            

# 결과그래프(loss, accuracy)를 그리는것은 한번 고민해보시기 바랍니다.!!

# 프로그램 수행환경 
# Server : AWS EC2 2.4.1
#          1 GPU(Tesla T4), 16GB GPU Memory
# CUDA : 11.0.4, cuDNN : 8
# Tensorflow-gpu : 2.4.1
 
# 최종 저장된 모델

# 5개의 TFRecord로 학습한 경우
# Epoch 00016: val_acc improved from 0.95300 to 0.95400, 
# saving model to ./checkpoint/cat-dog-000016-0.954000-1.000000.hdf5

# 1개의 TFRecord로 학습한 경우
# Epoch 00015: val_acc improved from 0.95140 to 0.95420, 
# saving model to ./checkpoint/cat-dog-000015-0.954200-0.995150.hdf5